# 04 - Vertex AI Custom Model - scikit-learn - в Notebook

Навчання моделі відбувається там, де вона споживає обчислювальні ресурси.  З Vertex AI у вас є вибір для налаштування обчислювальних ресурсів, доступних для навчання.  Цей ноутбук є прикладом середовища виконання.  Під час його налаштування можна було вибрати тип комп'ютера та прискорювачі (GPU).  

У цьому блокноті показано навчання моделі безпосередньо під час виконання в середовищі блокнота.  Потім модель зберігається і переміщується в GCS для розгортання в Vertex AI > Endpoint для онлайн-прогнозування.  Навчання моделі виконується за допомогою [scikit-learn](https://scikit-learn.org/stable/) і призначене для демонстрації стандартного підходу до логістичної регресії.

---
## Налаштування

вхідні дані:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
REGION = 'us-central1'
EXPERIMENT = '04'
SERIES = '04'

# джерело даних
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Ресурси
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'
TRAINING_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/sklearn-cpu.0-23:latest'

# Навчання моделі
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id-splits'


пакети:

In [ ]:
from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud import storage

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

import pickle
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import json

from datetime import datetime
import os

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


клієнти:

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project=PROJECT_ID)

параметри:

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"
BLOB = f"{SERIES}/{EXPERIMENT}/models/{TIMESTAMP}/model/model.pkl"

трекінг експеременту:

In [ ]:
FRAMEWORK = 'sklearn'
TASK = 'classification'
MODEL_TYPE = 'logistic-regression'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

оточення:

In [ ]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Налаштування Vertex AI Experiments

Код у цьому розділі ініціалізує експеримент і запускає прогін.  Впродовж усього блокнота в секціях для навчання моделі та оцінювання експерименту буде записано інформацію про експеримент за допомогою:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

Ініціалізація експерименту:

In [ ]:
aiplatform.init(experiment = EXPERIMENT_NAME)

Створення експерименту в Vertex AI Experiments:

In [ ]:
expRun = aiplatform.ExperimentRun.create(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

Логи параметрів:

In [ ]:
expRun.log_params({'experiment': EXPERIMENT, 'series': SERIES, 'project_id': PROJECT_ID})

---
## Навчальні дані
У цій вправі джерелом даних є таблиця в Google BigQuery.  Хоча можна перенести всю таблицю в локальний блокнот як фреймворк даних Pandas, це не є масштабованим рішенням для дуже великих навчальних таблиць.

### Схема даних
Використання BigQueries Information_Schema - це простий спосіб швидко отримати інформацію про стовпці наших навчальних даних.  У цьому випадку нам потрібні назви стовпців і типи даних, щоб налаштувати зчитування даних і вхідні дані моделі.  У цьому розділі ми отримаємо інформацію про стовпці для джерела навчальної таблиці.

In [ ]:
query = f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMNS` WHERE TABLE_NAME = '{BQ_TABLE}'"
schema = bq.query(query).to_dataframe()
schema

### Кількість класів для стовпця міток: VAR_TARGET
Це приклад керованого навчання, який класифікує приклади за класами, знайденими у стовпчику міток, що зберігається у змінній `VAR_TARGET`.

In [ ]:
nclasses = bq.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE {VAR_TARGET} is not null').to_dataframe()
nclasses

In [ ]:
nclasses = nclasses.shape[0]
nclasses

In [ ]:
expRun.log_params({'data_source': f'bq://{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}', 'nclasses': nclasses, 'var_split': 'splits', 'var_target': VAR_TARGET})

---
## Читаємо з BigQuery

In [ ]:
VAR_OMIT = (VAR_OMIT  + '-' + VAR_TARGET).split('-')

In [ ]:
train_query = f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'TRAIN'"
train = bq.query(train_query).to_dataframe()
X_train = train.loc[:, ~train.columns.isin(VAR_OMIT)]
y_train = train[VAR_TARGET].astype('int')

In [ ]:
val_query = f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'VALIDATE'"
val = bq.query(val_query).to_dataframe()
X_val = val.loc[:, ~val.columns.isin(VAR_OMIT)]
y_val = val[VAR_TARGET].astype('int')

In [ ]:
test_query = f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'TEST'"
test = bq.query(test_query).to_dataframe()
X_test = test.loc[:, ~test.columns.isin(VAR_OMIT)]
y_test = test[VAR_TARGET].astype('int')

---
## Тренування моделі у блокноті (локальний режим виконання)

У цьому прикладі використовується перехресна перевірка сіткового пошуку ([GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearch#sklearn.model_selection.GridSearchCV)) для тестування різних комбінацій параметрів моделі, щоб визначити найкращу модель логістичної регресії з оцінкою точності ([sklearn logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).  

Попередження Вибір алгоритму залежить від обраного штрафу. Підтримувані штрафи розв'язувачем:
- 'newton-cg' - ['l2', 'none'].
- 'lbfgs' - ['l2', 'none']
- 'liblinear' - ['l1', 'l2'].
- 'sag' - ['l2', 'none']
- 'saga' - ['elasticnet', 'l1', 'l2', 'none'].

In [ ]:
# визначення параметрів
solver = 'newton-cg'
penalty = 'l2'

# визначення моделі
logistic = LogisticRegression(solver=solver, penalty=penalty)

# Нормалізація даних
scaler = StandardScaler()


In [ ]:
expRun.log_params({'solver': solver, 'penalty': penalty})

In [ ]:
# ініціалізація пайплайну
pipe = Pipeline(steps=[("scaler", scaler), ("logistic", logistic)])

# визначення пошуку по сітці
model = pipe.fit(X_train, y_train)


In [ ]:
model.get_params

Оцінювання моделі

In [ ]:
y_pred_training = model.predict(X_train)
training_acc = metrics.accuracy_score(y_train, y_pred_training) 
training_prec = metrics.precision_score(y_train, y_pred_training)
training_rec = metrics.recall_score(y_train, y_pred_training)
training_rocauc = metrics.roc_auc_score(y_train, y_pred_training)
print('accuracy: ' + str(training_acc) + ', precision: ' + str(training_prec) + ', recall: ' + str(training_rec) + ', ROC AUC: ' + str(training_rocauc))

In [ ]:
expRun.log_metrics({'training_accuracy': training_acc, 'training_precision':training_prec, 'training_recall': training_rec, 'training_roc_auc': training_rocauc})

In [ ]:
y_pred_val = model.predict(X_val)
val_acc = metrics.accuracy_score(y_val, y_pred_val) 
val_prec = metrics.precision_score(y_val, y_pred_val)
val_rec = metrics.recall_score(y_val, y_pred_val)
val_rocauc = metrics.roc_auc_score(y_val, y_pred_val)
print('accuracy: ' + str(val_acc) + ', precision: ' + str(val_prec) + ', recall: ' + str(val_rec) + ', ROC AUC: ' + str(val_rocauc))

In [ ]:
expRun.log_metrics({'validation_accuracy': val_acc, 'validation_precision': val_prec, 'validation_recall': val_rec, 'validation_roc_auc': val_rocauc})

In [ ]:
y_pred = model.predict(X_test)
test_acc = metrics.accuracy_score(y_test, y_pred) 
test_prec = metrics.precision_score(y_test, y_pred)
test_rec = metrics.recall_score(y_test, y_pred)
test_rocauc = metrics.roc_auc_score(y_test, y_pred)
print('accuracy: ' + str(test_acc) + ', precision: ' + str(test_prec) + ', recall: ' + str(test_rec) + ', ROC AUC: ' + str(test_rocauc))

In [ ]:
expRun.log_metrics({'test_accuracy': test_acc, 'test_precision': test_prec, 'test_recall': test_rec, 'test_roc_auc': test_rocauc})

Створіть прогноз на основі батчу тестових даних і перегляньте матрицю плутанини:

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred) 

class_names = [0,1]
fig, ax = plt.subplots() 
tick_marks = np.arange(len(class_names)) 
plt.xticks(tick_marks, class_names) 
plt.yticks(tick_marks, class_names) 

# heatmap 
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu", fmt='g') 
ax.xaxis.set_label_position("top") 
plt.tight_layout() 
plt.title('Confusion matrix', y=1.1) 
plt.ylabel('Actual label') 
plt.xlabel('Predicted label')

### Збереження моделі

In [ ]:
# pickle
!mkdir model_artifacts

with open('model_artifacts/model.pkl','wb') as f:
    pickle.dump(model,f)

In [ ]:
# Завантажте модель до GCS
bucket = storage.Client().bucket(BUCKET)
blob = bucket.blob(BLOB)
blob.upload_from_filename('model_artifacts/model.pkl')

In [ ]:
# підтвердити, що модель знаходиться у правильному бакеті
!gsutil ls {URI}/models/{TIMESTAMP}/model/

In [ ]:
expRun.log_params({'model.save': f'{URI}/models/{TIMESTAMP}/model'})

### Завантаження моделі

In [ ]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Модель вже зареєстрована:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("Ця версія вже завантажена, ніяких дій не виконується.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Завантаження моделі як нової версії за замовчуванням.')
        parent_model = modelmatch[0].resource_name

else:
    print('Це нова модель, створена в реєстрі моделей')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{SERIES}_{EXPERIMENT}',
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model =  parent_model,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )

In [ ]:
print(f'Перегляньте модель у Реєстрі моделей ШІ Vertex:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}?project={PROJECT_ID}')

### Vertex AI Experiment оновлення та огляд

In [ ]:
expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.name': model.name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name
})

Завершіть експеримент:

In [ ]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Відновити експеримент:

In [ ]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [ ]:
exp.get_data_frame()

### Перегляд експерименту та запуск у консолі

In [ ]:
print(f'Перегляньте експеримент у консолі:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/experiments/{EXPERIMENT_NAME}?project={PROJECT_ID}')

In [ ]:
print(f'Перегляньте виконання експерименту у консолі:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/experiments/{EXPERIMENT_NAME}/runs/{EXPERIMENT_NAME}-{RUN_NAME}?project={PROJECT_ID}')

Отримайте список усіх експериментів у цьому проєкті:

In [ ]:
experiments = aiplatform.Experiment.list()

Видаліть експерименти, які не входять до SERIES:

In [ ]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

Об'єднайте прогони з усіх експериментів у SERIES в єдиний фрейм даних:

In [ ]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

Створіть ранги для моделей в межах експерименту та всієї SERIES:

In [ ]:
def ranker(metric = 'metric.test_roc_auc'):
    ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
    ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
    ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
    return ranks.sort_values(by = ['experiment_name', 'run_name'])
    
ranks = ranker('metric.test_roc_auc')
ranks

In [ ]:
current_rank = ranks.loc[(ranks['param.model.display_name'] == model.display_name) & (ranks['param.model.version_id'] == model.version_id)]
current_rank

In [ ]:
print(f"Поточна модель займає {current_rank['experiment_rank'].iloc[0]} у цьому експерименті та {current_rank['series_rank'].iloc[0]} у цій серії.")

### Створення Endpoint

In [ ]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint існує: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint створено: {endpoint.resource_name}")
    
print(f'Перегляд кінцевої точки в консолі:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

In [ ]:
endpoint.display_name

In [ ]:
endpoint.traffic_split

In [ ]:
deployed_models = endpoint.list_models()
deployed_models

### Чи варто розгортати цю модель?
Чи є вона кращою за модель, яка вже розгорнута на кінцевій точці?

In [ ]:
deploy = False
if deployed_models:
    for deployed_model in deployed_models:
        deployed_rank = ranks.loc[(ranks['param.model.display_name'] == deployed_model.display_name) & (ranks['param.model.version_id'] == deployed_model.model_version_id)]['series_rank'].iloc[0]
        model_rank = current_rank['series_rank'].iloc[0]
        if deployed_model.display_name == model.display_name and deployed_model.model_version_id == model.version_id:
            print(f'Поточна модель/версія вже розгорнута.')
            break
        elif model_rank <= deployed_rank:
            deploy = True
            print(f'Поточна модель вважається кращою ({model_rank}), ніж розгорнута модель ({deployed_rank}).')
            break
    if deploy == False: print(f'Поточна модель має гірший рейтинг ({model_rank}), ніж розгорнута модель ({deployed_rank})')
else: 
    deploy = True
    print('Наразі не розгорнуто жодної моделі.')

### Deploy Model To Endpoint

In [ ]:
if deploy:
    print(f'Розгортання моделі зі 100% трафіку...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )
else: print(f'Не розгортається - поточна модель гірша ({model_rank}) за розгорнуту модель ({deployed_rank})')

### Видалення розгорнутих моделей без трафіку

In [ ]:
for deployed_model in endpoint.list_models():
    if deployed_model.id в endpoint.traffic_split:
        print(f"Модель {deployed_model.display_name} з версією {deployed_model.model_version_id} має трафік = {endpoint.traffic_split[deployed_model.id]}")
    else:
        endpoint.undeploy(deployed_model_id = deployed_model.id)
        print(f"Розгортання {deployed_model.display_name} з версією {deployed_model.model_version_id}, оскільки вона не має трафіку.")

In [ ]:
endpoint.traffic_split

In [ ]:
endpoint.list_models()

---
## Прогнозування

### Підготуйте запис для прогнозування: список екземплярів

In [ ]:
instances = [X_test.to_dict(orient='split')['data'][0]]

In [ ]:
instances[0]

### Отримання прогнозів: Python Client

In [ ]:
prediction = endpoint.predict(instances=instances)
prediction

In [ ]:
prediction.predictions[0]

In [ ]:
np.argmax(prediction.predictions[0])

### Отримання прогнозів: REST

In [ ]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": instances}))

In [ ]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

### Отримання прогнозів: gcloud (CLI)

In [ ]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json